In [1]:
# We deliberately left a bug in the code for animating Duke. 
# If you click on one of the chessboard squares to the right of Duke, he waves anyway. 
# Why? Find a one-line fix for the bug.


import pygame

def draw_board(the_board):
    """Draw a chess board with queens, from the_board"""
    
    pygame.init()
    colors = [(255, 0, 0), (0, 0, 0)] # set colors for checks red, black
    
    my_clock = pygame.time.Clock()
    
    n = len(the_board) # this is an nxn chess board
    surface_sz = 480 # Proposed physical surface size.
    sq_sz = surface_sz // n # sq_sz is length of a square.
    surface_sz = n * sq_sz # adjust to exactly fit n squares
    
    # Create the surface of (width, height), ant it's window
    surface = pygame.display.set_mode((surface_sz, surface_sz))
    
    # load ball image
    ball = pygame.image.load("ball.png")
    
    # Load the sprite sheet
    duke_sprite_sheet = pygame.image.load("duke_spritesheet.png")
    
    # Instantiate two duke instances, put them on the chessboard
    duke1 = DukeSprite(duke_sprite_sheet, (sq_sz*2, 0))
    duke2 = DukeSprite(duke_sprite_sheet, (sq_sz*5, sq_sz))
    
    all_sprites = []      # Keep a list of all sprites in the game
    
    # Add them to the list of sprites which our game loop manages
    all_sprites.append(duke1)
    all_sprites.append(duke2)
    
    # Use extra offset to center the ball in its square.
    # If the square is too small, offset becomes negative,
    # but it will still be centered
    ball_offset = (sq_sz - ball.get_width()) // 2
    
    # Create a sprite object for each queen, and populate our list.
    for (col, row) in enumerate(the_board):
        a_queen = QueenSprite(ball,
                   (col*sq_sz+ball_offset, row*sq_sz+ball_offset))
        all_sprites.append(a_queen)

    
    while True:
        # Look for an event from keyboard, mouse etc
        ev = pygame.event.poll()
        if ev.type == pygame.QUIT:
            break;
            
        if ev.type == pygame.KEYDOWN:
            key = ev.dict["key"]
            if key == 27:                  # On Escape key ...
                break                      #   leave the game loop.
            if key == ord("r"):
                colors[0] = (255, 0, 0)    # Change to red + black.
            elif key == ord("g"):
                colors[0] = (0, 255, 0)    # Change to green + black.
            elif key == ord("b"):
                colors[0] = (0, 0, 255)    # Change to blue + black.

        if ev.type == pygame.MOUSEBUTTONDOWN:
            posn_of_click = ev.dict["pos"]
            for sprite in all_sprites:
                if sprite.contains_point(posn_of_click):
                    sprite.handle_click()
                    break           
        
            
        # Ask every sprite to update itself.
        for sprite in all_sprites:
            sprite.update()
    
        # Draw a fresh background ( a blank chess board)
        for row in range(n): # Draw each row of the board
            c_indx = row % 2 # Change strating color on each row
            for col in range(n): # Run through cols drawing squares
                the_square = (col*sq_sz, row*sq_sz, sq_sz, sq_sz)
                surface.fill(colors[c_indx], the_square)
                # now flip the color index for the next square
                c_indx = (c_indx + 1) % 2
                
        # Now that squares are drawn, draw the queens
        for (col, row) in enumerate(the_board):
            surface.blit(ball, (col*sq_sz + ball_offset, row*sq_sz + ball_offset))
            
        # Ask every sprite to draw itself.
        for sprite in all_sprites:
            sprite.draw(surface)
            
        my_clock.tick(60)  # Waste time so that frame rate becomes 60 fps
            
        pygame.display.flip()
    pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

gravity = 0.0001

class QueenSprite:
    
    def __init__(self, img, target_posn):
        self.image = img
        self.target_posn = target_posn
        (x, y) = target_posn
        self.posn = (x, 0) # Start ball at top of its column
        self.y_velocity = 0 # with zero initial velocity
        
    def update(self):
        self.y_velocity += gravity # gravity changes velocity
        (x, y) = self.posn
        new_y_pos = y + self.y_velocity # velocity moves the ball to its new position
        (target_x, target_y) = self.target_posn
        dist_to_go = target_y - new_y_pos
        
        if dist_to_go < 0:
            self.y_velocity = -0.65 * self.y_velocity
            new_y_pos = target_y + dist_to_go
            
        self.posn = (x, new_y_pos)
        
    def draw(self, target_surface): # same as before
        target_surface.blit(self.image, self.posn)
        
        
    def contains_point(self, pt):
        """ Return True if my sprite rectangle contains point pt """
        (my_x, my_y) = self.posn
        my_width = self.image.get_width()
        my_height = self.image.get_height()
        (x, y) = pt
        return ( x >= my_x and x < my_x + my_width and
               y >= my_y and y < my_y + my_height)
    
    def handle_click(self):
        self.y_velocity += -0.3   # Kick it up

        

In [3]:
class DukeSprite:
    def __init__(self, img, target_posn):
        self.image = img
        self.posn = target_posn
        self.anim_frame_count = 0
        self.curr_patch_num = 0
        
    def update(self):
        if self.anim_frame_count > 0:
            self.anim_frame_count = (self.anim_frame_count + 1) % 60
            self.curr_patch_num = self.anim_frame_count // 6
        
    
    def draw(self, target_surface):
        patch_rect = (self.curr_patch_num * 50, 0, 50, self.image.get_height())
        target_surface.blit(self.image, self.posn, patch_rect)
        
    
    def handle_click(self):
        if self.anim_frame_count == 0:
            self.anim_frame_count = 5
        
    
    def contains_point(self, pt):
        # Use code from QueensSprite here
        """ Return True if my sprite rectangle contains point pt """
        (my_x, my_y) = self.posn
        my_width = self.image.get_width()
        my_height = self.image.get_height()
        (x, y) = pt
        return ( x >= my_x and x < my_x + my_width and
               y >= my_y and y < my_y + my_height)

In [4]:
if __name__ == "__main__":
    draw_board([0, 5, 3, 1, 6, 4, 2])    # 7 x 7 to test window size
#     draw_board([6, 4, 2, 0, 5, 7, 1, 3])
#     draw_board([9, 6, 0, 3, 10, 7, 2, 4, 12, 8, 11, 5, 1])  # 13 x 13
#     draw_board([11, 4, 8, 12, 2, 7, 3, 15, 0, 14, 10, 6, 13, 1, 5, 9])